In [ ]:
from typing import Iterable, List, Union
import numpy as np
from citylearn.base import Environment
np.seterr(divide = 'ignore', invalid = 'ignore')
ZERO_DIVISION_CAPACITY = 0.00001

class Device(Environment):
    def __init__(self, efficiency: float = None, **kwargs):
        r"""Initialize `Device`.

        Parameters
        ----------
        efficiency : float, default: 1.0
            Technical efficiency. Must be set to > 0.

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(**kwargs)
        self.efficiency = efficiency

    @property
    def efficiency(self) -> float:
        """Technical efficiency."""

        return self.__efficiency

    @efficiency.setter
    def efficiency(self, efficiency: float):
        if efficiency is None:
            self.__efficiency = 1.0
        else:
            assert efficiency > 0, 'efficiency must be > 0.'
            self.__efficiency = efficiency

class ElectricDevice(Device):
    def __init__(self, nominal_power: float, **kwargs):
        r"""Initialize `Device`.

        Parameters
        ----------
        nominal_power : float
            Electric device nominal power >= 0. If == 0, set to 0.00001 to avoid `ZeroDivisionError`.

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(**kwargs)
        self.nominal_power = nominal_power

    @property
    def nominal_power(self) -> float:
        r"""Nominal power."""

        return self.__nominal_power

    @property
    def electricity_consumption(self) -> List[float]:
        r"""Electricity consumption time series."""

        return self.__electricity_consumption

    @property
    def available_nominal_power(self) -> float:
        r"""Difference between `nominal_power` and `electricity_consumption` at current `time_step`."""

        return None if self.nominal_power is None else self.nominal_power - self.electricity_consumption[self.time_step]

    @nominal_power.setter
    def nominal_power(self, nominal_power: float):
        if nominal_power is None or nominal_power == 0:
            self.__nominal_power = ZERO_DIVISION_CAPACITY
        else:
            assert nominal_power >= 0, 'nominal_power must be >= 0.'
            self.__nominal_power = nominal_power

    def update_electricity_consumption(self, electricity_consumption: float):
        r"""Updates `electricity_consumption` at current `time_step`.
        
        Parameters
        ----------
        electricity_consumption : float
            value to add to current `time_step` `electricity_consumption`. Must be >= 0.
        """

        assert electricity_consumption >= 0, 'electricity_consumption must be >= 0.'
        self.__electricity_consumption[self.time_step] += electricity_consumption

    def next_time_step(self):
        r"""Advance to next `time_step` and set `electricity_consumption` at new `time_step` to 0.0."""

        super().next_time_step()
        self.__electricity_consumption.append(0.0)

    def reset(self):
        r"""Reset `ElectricDevice` to initial state and set `electricity_consumption` at `time_step` 0 to = 0.0."""

        super().reset()
        self.__electricity_consumption = [0.0]

class HeatPump(ElectricDevice):
    def __init__(self, nominal_power: float, efficiency: float = None, target_heating_temperature: float = None, target_cooling_temperature: float = None, **kwargs):
        r"""Initialize `HeatPump`.

        Parameters
        ----------
        nominal_power: float
            Maximum amount of electric power that the heat pump can consume from the power grid (given by the nominal power of the compressor).
        efficiency : float, default: 0.2
            Technical efficiency.
        target_heating_temperature : float, default: 45.0
            Target heating supply dry bulb temperature in [C].
        target_cooling_temperature : float, default: 8.0
            Target cooling supply dry bulb temperature in [C].

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(nominal_power = nominal_power, efficiency = efficiency, **kwargs)
        self.target_heating_temperature = target_heating_temperature
        self.target_cooling_temperature = target_cooling_temperature

    @property
    def target_heating_temperature(self) -> float:
        r"""Target heating supply dry bulb temperature in [C]."""

        return self.__target_heating_temperature

    @property
    def target_cooling_temperature(self) -> float:
        r"""Target cooling supply dry bulb temperature in [C]."""

        return self.__target_cooling_temperature

    @target_heating_temperature.setter
    def target_heating_temperature(self, target_heating_temperature: float):
        if target_heating_temperature is None:
            self.__target_heating_temperature = 45.0
        else:
            self.__target_heating_temperature = target_heating_temperature

    @target_cooling_temperature.setter
    def target_cooling_temperature(self, target_cooling_temperature: float):
        if target_cooling_temperature is None:
            self.__target_cooling_temperature = 8.0
        else:
            self.__target_cooling_temperature = target_cooling_temperature

    @ElectricDevice.efficiency.setter
    def efficiency(self, efficiency: float):
        efficiency = 0.2 if efficiency is None else efficiency
        ElectricDevice.efficiency.fset(self, efficiency)

    def get_cop(self, outdoor_dry_bulb_temperature: Union[float, Iterable[float]], heating: bool) -> Union[float, Iterable[float]]:
        r"""Return coefficient of performance.

        Calculate the Carnot cycle COP for heating or cooling mode. COP is set to 20 if < 0 or > 20.

        Parameters
        ----------
        outdoor_dry_bulb_temperature : Union[float, Iterable[float]]
            Outdoor dry bulb temperature in [C].
        heating : bool
            If `True` return the heating COP else return cooling COP.

        Returns
        -------
        cop : Union[float, Iterable[float]]
            COP as single value or time series depending on input parameter types.

        Notes
        -----
        heating_cop = (`t_target_heating` + 273.15)*`efficiency`/(`t_target_heating` - outdoor_dry_bulb_temperature)
        cooling_cop = (`t_target_cooling` + 273.15)*`efficiency`/(outdoor_dry_bulb_temperature - `t_target_cooling`)
        """

        c_to_k = lambda x: x + 273.15
        outdoor_dry_bulb_temperature = np.array(outdoor_dry_bulb_temperature)

        if heating:
            cop = self.efficiency*c_to_k(self.target_heating_temperature)/(self.target_heating_temperature - outdoor_dry_bulb_temperature)
        else:
            cop = self.efficiency*c_to_k(self.target_cooling_temperature)/(outdoor_dry_bulb_temperature - self.target_cooling_temperature)
        
        cop = np.array(cop)
        cop[cop < 0] = 20
        cop[cop > 20] = 20
        return cop

    def get_max_output_power(self, outdoor_dry_bulb_temperature: Union[float, Iterable[float]], heating: bool, max_electric_power: Union[float, Iterable[float]] = None) -> Union[float, Iterable[float]]:
        r"""Return maximum output power.

        Calculate maximum output power from heat pump given `cop`, `available_nominal_power` and `max_electric_power` limitations.

        Parameters
        ----------
        outdoor_dry_bulb_temperature : Union[float, Iterable[float]]
            Outdoor dry bulb temperature in [C].
        heating : bool
            If `True` use heating COP else use cooling COP.
        max_electric_power : Union[float, Iterable[float]], optional
            Maximum amount of electric power that the heat pump can consume from the power grid.

        Returns
        -------
        max_output_power : Union[float, Iterable[float]]
            Maximum output power as single value or time series depending on input parameter types.

        Notes
        -----
        max_output_power = min(max_electric_power, `available_nominal_power`)*cop
        """

        cop = self.get_cop(outdoor_dry_bulb_temperature, heating)

        if max_electric_power is None: 
            return self.available_nominal_power*cop  
        else:
            return np.minimum(max_electric_power, self.available_nominal_power)*cop

    def get_input_power(self, output_power: Union[float, Iterable[float]], outdoor_dry_bulb_temperature: Union[float, Iterable[float]], heating: bool) -> Union[float, Iterable[float]]:
        r"""Return input power.

        Calculate power needed to meet `output_power` given `cop` limitations.

        Parameters
        ----------
        output_power : Union[float, Iterable[float]]
            Output power from heat pump
        outdoor_dry_bulb_temperature : Union[float, Iterable[float]]
            Outdoor dry bulb temperature in [C].
        heating : bool
            If `True` use heating COP else use cooling COP.

        Returns
        -------
        input_power : Union[float, Iterable[float]]
            Input power as single value or time series depending on input parameter types.

        Notes
        -----
        input_power = output_power/cop
        """

        return output_power/self.get_cop(outdoor_dry_bulb_temperature, heating)

    def autosize(self, outdoor_dry_bulb_temperature: Iterable[float], cooling_demand: Iterable[float] = None, heating_demand: Iterable[float] = None, safety_factor: float = None):
        r"""Autosize `nominal_power`.

        Set `nominal_power` to the minimum power needed to always meet `cooling_demand` + `heating_demand`.

        Parameters
        ----------
        outdoor_dry_bulb_temperature : Union[float, Iterable[float]]
            Outdoor dry bulb temperature in [C].
        cooling_demand : Union[float, Iterable[float]], optional
            Cooling demand in [kWh].
        heating_demand : Union[float, Iterable[float]], optional
            Heating demand in [kWh].
        safety_factor : float, default: 1.0
            `nominal_power` is oversized by factor of `safety_factor`.

        Notes
        -----
        `nominal_power` = max((cooling_demand/cooling_cop) + (heating_demand/heating_cop))*safety_factor
        """
        
        safety_factor = 1.0 if safety_factor is None else safety_factor

        if cooling_demand is not None:
            cooling_nominal_power = np.array(cooling_demand)/self.get_cop(outdoor_dry_bulb_temperature, False)
        else:
            cooling_nominal_power = 0
        
        if heating_demand is not None:
            heating_nominal_power = np.array(heating_demand)/self.get_cop(outdoor_dry_bulb_temperature, True)
        else:
            heating_nominal_power = 0

        self.nominal_power = np.nanmax(cooling_nominal_power + heating_nominal_power)*safety_factor

class ElectricHeater(ElectricDevice):
    def __init__(self, nominal_power: float, efficiency: float = None, **kwargs):
        r"""Initialize `ElectricHeater`.

        Parameters
        ----------
        nominal_power : float
            Maximum amount of electric power that the electric heater can consume from the power grid.
        efficiency : float, default: 0.9
            Technical efficiency.

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(nominal_power = nominal_power, efficiency = efficiency, **kwargs)

    @ElectricDevice.efficiency.setter
    def efficiency(self, efficiency: float):
        efficiency = 0.9 if efficiency is None else efficiency   
        ElectricDevice.efficiency.fset(self, efficiency)

    def get_max_output_power(self, max_electric_power: Union[float, Iterable[float]] = None) -> Union[float, Iterable[float]]:
        r"""Return maximum output power.

        Calculate maximum output power from heat pump given `max_electric_power` limitations.

        Parameters
        ----------
        max_electric_power : Union[float, Iterable[float]], optional
            Maximum amount of electric power that the heat pump can consume from the power grid.

        Returns
        -------
        max_output_power : Union[float, Iterable[float]]
            Maximum output power as single value or time series depending on input parameter types.

        Notes
        -----
        max_output_power = min(max_electric_power, `available_nominal_power`)*`efficiency`
        """

        if max_electric_power is None:
            return self.available_nominal_power*self.efficiency
        else:
            return np.min(max_electric_power, self.available_nominal_power)*self.efficiency

    def get_input_power(self, output_power: Union[float, Iterable[float]]) -> Union[float, Iterable[float]]:
        r"""Return input power.

        Calculate power demand to meet `output_power`.

        Parameters
        ----------
        output_power : Union[float, Iterable[float]] 
            Output power from heat pump

        Returns
        -------
        input_power : Union[float, Iterable[float]]
            Input power as single value or time series depending on input parameter types.

        Notes
        -----
        input_power = output_power/`efficiency`
        """

        return np.array(output_power)/self.efficiency

    def autosize(self, demand: Iterable[float], safety_factor: float = None):
        r"""Autosize `nominal_power`.

        Set `nominal_power` to the minimum power needed to always meet `demand`.

        Parameters
        ----------
        demand : Union[float, Iterable[float]], optional
            Heating emand in [kWh].
        safety_factor : float, default: 1.0
            `nominal_power` is oversized by factor of `safety_factor`.

        Notes
        -----
        `nominal_power` = max(demand/`efficiency`)*safety_factor
        """

        safety_factor = 1.0 if safety_factor is None else safety_factor
        self.nominal_power = np.nanmax(np.array(demand)/self.efficiency)*safety_factor

class PV(ElectricDevice):
    def __init__(self, nominal_power: float, **kwargs):
        r"""Initialize `PV`.

        Parameters
        ----------
        nominal_power : float
            PV array output power in [kW]. Must be >= 0.

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(nominal_power=nominal_power,**kwargs)


    def get_generation(self, inverter_ac_power_per_kw: Union[float, Iterable[float]]) -> Union[float, Iterable[float]]:
        r"""Get solar generation output.

        Parameters
        ----------
        inverter_ac_power_perk_w : Union[float, Iterable[float]]
            Inverter AC power output per kW of PV capacity in [W/kW].

        Returns
        -------
        generation : Union[float, Iterable[float]]
            Solar generation as single value or time series depending on input parameter types.

        Notes
        -----
        .. math::
            \textrm{generation} = \frac{\textrm{capacity} \times \textrm{inverter_ac_power_per_w}}{1000}
        """

        return self.nominal_power*np.array(inverter_ac_power_per_kw)/1000

    def autosize(self, demand: Iterable[float], safety_factor: float = None):
        r"""Autosize `nominal_power`.

        Set `nominal_power` to the minimum nominal_power needed to always meet `demand`.

        Parameters
        ----------
        demand : Union[float, Iterable[float]], optional
            Heating emand in [kWh].
        safety_factor : float, default: 1.0
            The `nominal_power` is oversized by factor of `safety_factor`.

        Notes
        -----
        `nominal_power` = max(demand/`efficiency`)*safety_factor
        """

        safety_factor = 1.0 if safety_factor is None else safety_factor
        self.nominal_power = np.nanmax(np.array(demand)/self.efficiency)*safety_factor

class StorageDevice(Device):
    def __init__(self, capacity: float, efficiency: float = None, loss_coefficient: float = None, initial_soc: float = None, efficiency_scaling: float = None, **kwargs):
        r"""Initialize `StorageDevice`.

        Parameters
        ----------
        capacity : float
            Maximum amount of energy the storage device can store in [kWh]. Must be >= 0 and if == 0 or None, set to 0.00001 to avoid `ZeroDivisionError`.
        efficiency : float, default: 0.9
            Technical efficiency.
        loss_coefficient : float, default: 0.006
            Standby hourly losses. Must be between 0 and 1 (this value is often 0 or really close to 0).
        initial_soc : float, default: 0.0
            State of charge when `time_step` = 0. Must be >= 0 and < `capacity`.
        efficiency_scaling : float, default: 0.5
            `efficiency` exponent scaling for `efficienct` such that `efficiency` **= `efficiency_scaling`

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        self.efficiency_scaling = efficiency_scaling
        self.capacity = capacity
        self.loss_coefficient = loss_coefficient
        self.initial_soc = initial_soc
        super().__init__(efficiency = efficiency, **kwargs)

    @property
    def capacity(self) -> float:
        r"""Maximum amount of energy the storage device can store in [kWh]."""

        return self.__capacity

    @property
    def loss_coefficient(self) -> float:
        r"""Standby hourly losses."""

        return self.__loss_coefficient

    @property
    def initial_soc(self) -> float:
        r"""State of charge when `time_step` = 0 in [kWh]."""

        return self.__initial_soc

    @property
    def soc(self) -> List[float]:
        r"""State of charge time series in [kWh]."""

        return self.__soc

    @property
    def soc_init(self) -> float:
        r"""Latest state of charge after accounting for standby hourly lossses."""

        return self.__soc[-1]*(1 - self.loss_coefficient)

    @property
    def efficiency_scaling(self) -> float:
        r"""`efficiency` exponent scaling."""

        return self.__efficiency_scaling

    @property
    def energy_balance(self) -> List[float]:
        r"""Charged/discharged energy time series in [kWh]."""

        return self.__energy_balance

    @Device.efficiency.setter
    def efficiency(self, efficiency: float):
        efficiency = 0.9 if efficiency is None else efficiency
        Device.efficiency.fset(self, efficiency**self.efficiency_scaling)

    @capacity.setter
    def capacity(self, capacity: float):
        if capacity is None or capacity == 0:
            self.__capacity = ZERO_DIVISION_CAPACITY
        else:
            assert capacity >= 0, 'capacity must be >= 0.'
            self.__capacity = capacity

    @loss_coefficient.setter
    def loss_coefficient(self, loss_coefficient: float):
        if loss_coefficient is None:
            self.__loss_coefficient = 0.006
        else:
            assert 0 <= loss_coefficient <= 1, 'initial_soc must be >= 0 and <= 1.'
            self.__loss_coefficient = loss_coefficient

    @initial_soc.setter
    def initial_soc(self, initial_soc: float):
        if initial_soc is None:
            self.__initial_soc = 0
        else:
            assert 0 <= initial_soc <= self.capacity, 'initial_soc must be >= 0 and <= capacity.'
            self.__initial_soc = initial_soc

    @efficiency_scaling.setter
    def efficiency_scaling(self, efficiency_scaling: float):
        if efficiency_scaling is None:
            self.__efficiency_scaling = 0.5
        else:
            self.__efficiency_scaling = efficiency_scaling

    def charge(self, energy: float):
        """Charges or discharges storage with respect to specified energy while considering `capacity` and `soc_init` limitations and, energy losses to the environment quantified by `efficiency`.

        Parameters
        ----------
        energy : float
            Energy to charge if (+) or discharge if (-) in [kWh].

        Notes
        -----
        If charging, soc = min(`soc_init` + energy*`efficiency`, `capacity`)
        If discharging, soc = max(0, `soc_init` + energy/`efficiency`)
        """
        
        # The initial State Of Charge (SOC) is the previous SOC minus the energy losses
        soc = min(self.soc_init + energy*self.efficiency, self.capacity) if energy >= 0 else max(0, self.soc_init + energy/self.efficiency)
        self.__soc.append(soc)
        self.__energy_balance.append(self.set_energy_balance())

    def set_energy_balance(self) -> float:
        r"""Calculate energy balance

        The energy balance is a derived quantity and is the product or quotient of the difference between consecutive SOCs and `efficiency`
        for discharge or charge events respectively thus, thus accounts for energy losses to environment during charging and discharge.
        """

        # actual energy charged/discharged irrespective of what is determined in the step function after 
        # taking into account storage design limits e.g. maximum power input/output, capacity
        previous_soc = self.initial_soc if self.time_step == 0 else self.soc[-2]
        energy_balance = self.soc[-1] - previous_soc*(1.0 - self.loss_coefficient)
        energy_balance = energy_balance/self.efficiency if energy_balance >= 0 else energy_balance*self.efficiency
        return energy_balance

    def autosize(self, demand: Iterable[float], safety_factor: float = None):
        r"""Autosize `capacity`.

        Set `capacity` to the minimum capacity needed to always meet `demand`.

        Parameters
        ----------
        demand : Union[float, Iterable[float]], optional
            Heating emand in [kWh].
        safety_factor : float, default: 1.0
            The `capacity` is oversized by factor of `safety_factor`.

        Notes
        -----
        `capacity` = max(demand/`efficiency`)*safety_factor
        """

        safety_factor = 1.0 if safety_factor is None else safety_factor
        self.capacity = np.nanmax(demand)*safety_factor

    def reset(self):
        r"""Reset `StorageDevice` to initial state."""

        super().reset()
        self.__soc = [self.initial_soc]
        self.__energy_balance = [0.0]

class StorageTank(StorageDevice):
    def __init__(self, capacity: float, max_output_power: float = None, max_input_power: float = None, **kwargs):
        r"""Initialize `StorageTank`.

        Parameters
        ----------
        capacity : float
            Maximum amount of energy the storage device can store in [kWh]. Must be >= 0 and if == 0 or None, set to 0.00001 to avoid `ZeroDivisionError`.
        max_output_power : float, optional
            Maximum amount of power that the storage unit can output [kW].
        max_input_power : float, optional
            Maximum amount of power that the storage unit can use to charge [kW].
        
        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super class.
        """

        super().__init__(capacity = capacity, **kwargs)
        self.max_output_power = max_output_power
        self.max_input_power = max_input_power

    @property
    def max_output_power(self) -> float:
        r"""Maximum amount of power that the storage unit can output [kW]."""

        return self.__max_output_power

    @property
    def max_input_power(self) -> float:
        r"""Maximum amount of power that the storage unit can use to charge [kW]."""

        return self.__max_input_power

    @max_output_power.setter
    def max_output_power(self, max_output_power: float):
        assert max_output_power is None or max_output_power >= 0, '`max_output_power` must be >= 0.'
        self.__max_output_power = max_output_power

    @max_input_power.setter
    def max_input_power(self, max_input_power: float):
        assert max_input_power is None or max_input_power >= 0, '`max_input_power` must be >= 0.'
        self.__max_input_power = max_input_power

    def charge(self, energy: float):
        """Charges or discharges storage with respect to specified energy while considering `capacity` and `soc_init` limitations and, energy losses to the environment quantified by `efficiency`.

        Parameters
        ----------
        energy : float
            Energy to charge if (+) or discharge if (-) in [kWh].

        Notes
        -----
        If charging, soc = min(`soc_init` + energy*`efficiency`, `max_input_power`, `capacity`)
        If discharging, soc = max(0, `soc_init` + energy/`efficiency`, `max_output_power`)
        """

        if energy >= 0:    
            energy = energy if self.max_input_power is None else np.nanmin([energy, self.max_input_power])
        else:
            energy = energy if self.max_output_power is None else np.nanmax([-self.max_output_power, energy])
        
        super().charge(energy)


In [4]:
class EVBattery(ElectricDevice, StorageDevice):
    def __init__(self, capacity: float, nominal_power: float, efficiency_curve: List[List[float]] = None, capacity_curve: List[List[float]] = None, **kwargs):
        r"""Initialize `EV`.

        Parameters
        ----------
        capacity : float
            Maximum amount of energy the storage device can store in [kWh]. Must be >= 0 and if == 0 or None, set to 0.00001 to avoid `ZeroDivisionError`.
        nominal_power: float
            Maximum amount of electric power that the EV batteries can use to charge or discharge.
        capacity_loss_coefficient : float, default: 0.00001
            Battery degradation; storage capacity lost in each charge and discharge cycle (as a fraction of the total capacity).
        efficiency_curve: list, default: [[0, 0.83],[0.3, 0.83],[0.7, 0.9],[1, 0.85]]
            Charging/Discharging efficiency as a function of the power released or consumed.
        capacity_curve: list, default: [[0.1, 1],[0.8, 1],[1.0, 0.2]]   
            Maximum power of the battery as a function of its current state of charge.

        Other Parameters
        ----------------
        **kwargs : dict
            Other keyword arguments used to initialize super classes.
        """
        super().__init__(capacity=capacity, nominal_power=nominal_power, **kwargs)
        self.efficiency_curve = efficiency_curve
        self.capacity_curve = capacity_curve
        
    @StorageDevice.capacity.getter
    def capacity(self) -> float:
        r"""Current time step maximum amount of energy the storage device can store in [kWh]"""

        return self.capacity[-1]

    @StorageDevice.efficiency.getter
    def efficiency(self) -> float:
        """Current time step technical efficiency."""

        return self.efficiency[-1]

    @ElectricDevice.electricity_consumption.getter
    def electricity_consumption(self) -> List[float]:
        r"""Electricity consumption time series."""

        return self.energy_balance

    @property
    def efficiency_curve(self) -> np.ndarray:
        """Charging/Discharging efficiency as a function of the power released or consumed."""

        return self.__efficiency_curve

    @property
    def capacity_curve(self) -> np.ndarray:
        """Maximum power of the battery as a function of its current state of charge."""

        return self.__capacity_curve

    
    @capacity.setter
    def capacity(self, capacity: float):
        capacity = ZERO_DIVISION_CAPACITY if capacity is None or capacity == 0 else capacity
        StorageDevice.capacity.fset(self, capacity)
        
        self.__capacity.append(capacity)

    @efficiency.setter
    def efficiency(self, efficiency: float):
        efficiency = 0.9 if efficiency is None else efficiency
        StorageDevice.efficiency.fset(self, efficiency)
        
        self.__efficiency.append(efficiency)


    @efficiency_curve.setter
    def efficiency_curve(self, efficiency_curve: List[List[float]]):
        if efficiency_curve is None:
            efficiency_curve = [[0, 0.83],[0.3, 0.83],[0.7, 0.9],[1, 0.85]]
        else:
            pass

        self.__efficiency_curve = np.array(efficiency_curve).T

    @capacity_curve.setter
    def capacity_curve(self, capacity_curve: List[List[float]]):
        if capacity_curve is None:
            capacity_curve = [[0.1, 1],[0.8, 1],[1.0, 0.2]]
        else:
            pass

        self.__capacity_curve = np.array(capacity_curve).T

    

    
    def simulate_EV_Energy(self, energy: float, mode, timestep):
        dt = 15 / 60  # 60 minute time intervals
        T0 = 0  # from 12 am to 12 am

        N_EVs = 5  # number of EVs
        DisCoun_EV = np.zeros(N_EVs)
        ELeaving_EV = np.zeros(N_EVs)
        Dislim_EV = 2  # Max No of discharge times

        # EV arrival & departure times and energy levels on arrival
        E0_EVs = 0.8*self.capacity* np.random.uniform(0.2, 0.6, N_EVs)
        ta_EVs = np.random.randint(int(7 / dt), int(10 / dt), N_EVs) - int(T0 / dt)
        td_EVs = np.random.randint(int(15 / dt), int(18 / dt), N_EVs) - int(T0 / dt)
        EV_energy = E0_EVs

        t = timestep

        for j in range(N_EVs):

            if (ta_EVs[j] <= t) and (td_EVs[j] > t):
                ELeaving_EV[j] = EV_energy[j] + self.capacity * self.efficiency[int(EV_energy[j])/self.capacity] * (td_EVs[j] - t) * (dt)
                if ELeaving_EV[j] > 0.8*self.capacity:
                    ELeaving_EV[j] = 0.8*self.capacity

                if mode == 'discharge':
                    if (EV_energy[j] > 0.1*self.capacity) and (energy > 0) and (DisCoun_EV[j] <= Dislim_EV) and (ELeaving_EV[j] > 0.1*self.capacity + E0_EVs[j]):
                        EV_energy[j] -= self.capacity / self.eff_EV[int(EV_energy[j]/self.capacity)] * (dt)
                        energy -= EV_energy
                        DisCoun_EV[j] += 1

                        if energy < 0:
                            EV_energy[j] -= energy
                            energy = 0

                        if EV_energy[j] < 0.1*self.capacity:
                            EV_energy[j] = 0.1*self.capacity
                    else:
                        mode = 'neutral'

                elif mode == 'charge':
                    EV_energy += self.capacity * self.eff_EV[int(EV_energy[j]/self.capacity)] * (dt)
                    energy += EV_energy

                    if EV_energy[j] > 0.8*self.capacity:
                        EV_energy[j] = 0.8*self.capacity

                else:
                    EV_energy[j] = EV_energy[j]
                    energy = energy

            else:
                EV_energy[j] = 0

        Load = energy
      
        return Load, EV_energy

  

    def reset(self):
        r"""Reset `EV Battery` to initial state."""

        super().reset()
        self.__efficiency_history = self.__efficiency[0:1]
        self.__capacity_history = self.__capacity[0:1]


NameError: name 'ElectricDevice' is not defined